# Multi-core processing in python
Here I'm using the multiprocessing module with `pool.map` and `pool.starmap` 
to run a function over a range of inputs using multiple cores at the same time.
The syntax is like a vectorised form of `parpool` in MATLAB.

## Serial vs parallel using a function with one argument

In [31]:
import time
import multiprocessing as mp

In [80]:
def double_wait(number):
    """Double a number, and wait a couple of secs"""
    double = number * 2
    time.sleep(1)
    return double

In [71]:
def time_it(func):
    """A decorator function to time execution"""
    def wrapper(*args):
        st = time.time()
        func(*args)
        et = time.time()
        elapsed_time = et - st
        print('Execution time:', elapsed_time, 'seconds')
        return func(*args)
    return wrapper

Normal, single-threaded loop

In [78]:
@time_it
def single_thread_one_arg(numbers):
    """Run double_wait in a serial loop"""
    doubles = []
    for num in numbers:
        doubles.append(double_wait(num))
    return doubles

In [76]:
numbers = [1, 2, 3, 4, 5, 6, 7, 8]
st_result = single_thread_one_arg(numbers)
print(st_result)

Execution time: 8.008411169052124 seconds
[2, 4, 6, 8, 10, 12, 14, 16]


Now using a parallel pool

In [79]:
@time_it
def multi_thread(numbers):
    """Run double_wait in a parallel loop"""
    pool = mp.Pool(mp.cpu_count())
    doubles = pool.map(double_wait, numbers)
    return doubles

In [75]:
mt_result = multi_thread(numbers)
print(mt_result)

Execution time: 1.0622458457946777 seconds
[2, 4, 6, 8, 10, 12, 14, 16]


## What about multiple arguments?

In [87]:
def two_args_wait(number, string):
    """Double a number, double a string, then wait"""
    double_num = number * 2
    double_str = string + string
    time.sleep(1)
    return double_num, double_str

In [93]:
@time_it
def single_thread_two_args(numbers, strings):
    """Run two_args_wait in a serial loop"""
    results = []
    for num, str in zip(numbers, strings):
        results.append(two_args_wait(num, str))
    return results

In [94]:
strings = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

In [99]:
serial_results =  single_thread_two_args(numbers, strings)
print(serial_results)

Execution time: 8.009098529815674 seconds
[(2, 'aa'), (4, 'bb'), (6, 'cc'), (8, 'dd'), (10, 'ee'), (12, 'ff'), (14, 'gg'), (16, 'hh')]


In [104]:
@time_it
def multi_thread_two_args(numbers, strings):
    """Run two_args_wait in a parallel loop"""
    pool = mp.Pool(mp.cpu_count())
    tuple_list = [(num, str) for num, str in zip(numbers, strings)]
    results = pool.starmap(two_args_wait, tuple_list)
    return results

In [105]:
parallel_results = multi_thread_two_args(numbers, strings)
print(parallel_results)

Execution time: 1.0460655689239502 seconds
[(2, 'aa'), (4, 'bb'), (6, 'cc'), (8, 'dd'), (10, 'ee'), (12, 'ff'), (14, 'gg'), (16, 'hh')]
